## Importing libraries

In [37]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

## Reading input from file

In [38]:
file = open('trainNN.txt')

lines = file.readlines()

X = []
labels = []

for line in lines:
    var = [float(x) for x in line.split()]
    X.append(var[:-1])
    labels.append(int(var[-1]))
    
number_of_classes = max(labels)
N = len(X)

Y = np.zeros((N,number_of_classes))

for i in range(N):
    Y[i,labels[i]-1] = 1
    
X = np.array(X) 
Y = np.array(Y)

In [39]:
X

array([[ 9.21323266, 11.82445528, 16.69098092, 19.56967227],
       [16.10344524, 23.89625138, 32.17012852, 39.75094393],
       [31.28930267, 47.86461292, 64.21983155, 79.74692811],
       ...,
       [ 7.90082578, 11.96311621, 16.30620007, 19.39478341],
       [15.07843625, 24.22942082, 31.61172694, 39.75461333],
       [32.27519136, 48.00916618, 64.0824622 , 80.59215177]])

In [40]:
Y

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       ...,
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.]])

### Number of layers = all layers except the input layers

### 2 Layer NN -> 1 Hidden Layer and 1 Output Layer

### First work with 2 Layer NN then we will extend this NN


In [41]:
L = 2 # number of layers

k = [4,Y.shape[1]]

W = []

for i in range(L):
    if(i==0):
        W.append(np.random.uniform(-1,1,(k[i],X.shape[1]+1)))
    else:
        W.append(np.random.uniform(-1,1,(k[i],k[i-1]+1)))

In [42]:
def sigmoid(z):
    return 1/(1+np.exp(z))

def derivative(z):
    return sigmoid(z)*(1-sigmoid(z))

In [43]:
W # random weights

[array([[-0.25091976,  0.90142861,  0.46398788,  0.19731697, -0.68796272],
        [-0.68801096, -0.88383278,  0.73235229,  0.20223002,  0.41614516],
        [-0.95883101,  0.9398197 ,  0.66488528, -0.57532178, -0.63635007],
        [-0.63319098, -0.39151551,  0.04951286, -0.13610996, -0.41754172]]),
 array([[ 0.22370579, -0.72101228, -0.4157107 , -0.26727631, -0.08786003],
        [ 0.57035192, -0.60065244,  0.02846888,  0.18482914, -0.90709917],
        [ 0.2150897 , -0.65895175, -0.86989681,  0.89777107,  0.93126407],
        [ 0.6167947 , -0.39077246, -0.80465577,  0.36846605, -0.11969501]])]

In [44]:
for epoch in range(5000):
    correct = 0
    for i in range(N):
        #forward propagation
        y_r = np.hstack((np.ones(1),X[i]))
        y = []
        y.append(y_r)
        v = []
        for r in range(L):
            v_r = np.zeros(k[r])
            for j in range(k[r]):
                v_r[j] = np.dot(W[r][j],y_r)
            v.append(v_r)
            y_r = np.hstack((np.ones(1),sigmoid(v_r)))
            y.append(y_r)
        
        # cost calculation
        #J += np.sum((y[L][1:]-Y[i])**2,axis=0)
            
        #back propagation
        
        if(np.argmax(y[L][1:])==np.argmax(Y[i])):
            correct += 1
        
        delta_L = (Y[i]-y[L][1:])*derivative(v[L-1])
        
        delta = []

        for r in range(L):
            if(r==L-1):
                delta.append(delta_L)
            else:
                delta.append(np.zeros(k[r]))

        for r in range(L-1):
            delta_r = []
            for j in range(k[L-r-2]):
                total = 0
                for p in range(k[L-r-1]):
                    total += delta[L-r-1][p]*W[L-r-1][p][j]
                delta_r.append(total*derivative(v[L-r-2][j]))

            delta[L-r-2] = np.array(delta_r)


        y = y[:-1]

        for r in range(L):
            for j in range(k[r]):
                W[r][j] = W[r][j] - 0.025*(delta[r][j]*y[r])
    
    accuracy = (correct/N)*100
    
    if(epoch%100==0):
        print('Epoch ',epoch,': ',accuracy)
    if(accuracy > 85):
        print('Epoch ',epoch,': ',accuracy)
        break

Epoch  0 :  22.2
Epoch  100 :  47.4
Epoch  200 :  51.800000000000004
Epoch  300 :  55.400000000000006
Epoch  400 :  52.2
Epoch  500 :  55.800000000000004
Epoch  600 :  63.800000000000004
Epoch  700 :  76.2
Epoch  800 :  78.4
Epoch  848 :  89.8


In [45]:
file = open('testNN.txt')

lines = file.readlines()

X = []
labels = []

for line in lines:
    var = [float(x) for x in line.split()]
    X.append(var[:-1])
    labels.append(int(var[-1]))
    
number_of_classes = max(labels)
N = len(X)

Y = np.zeros((N,number_of_classes))

for i in range(N):
    Y[i,labels[i]-1] = 1
    
X = np.array(X) 
Y = np.array(Y)

In [46]:
accuracy = 0
for i in range(N):
    #forward propagation
    y_r = np.hstack((np.ones(1),X[i]))
    y = []
    y.append(y_r)
    v = []
    for r in range(L):
        v_r = np.zeros(k[r])
        for j in range(k[r]):
            v_r[j] = np.dot(W[r][j],y_r)
        v.append(v_r)
        y_r = np.hstack((np.ones(1),sigmoid(v_r)))
        y.append(y_r)
    
    if(np.argmax(y[L][1:])==np.argmax(Y[i])):
        accuracy += 1

print("Test Accuracy: ",(accuracy/len(X))*100)

Test Accuracy:  88.0


In [47]:
W

[array([[-17.66693344,   0.51586933,   0.28047375,   0.26137091,
          -0.25632396],
        [ -0.6859551 ,  -0.86711054,   0.75662744,   0.23513546,
           0.45704539],
        [  0.2707426 ,   0.67583899,   0.31488573,  -0.34662097,
          -0.28160285],
        [ -0.63393708,  -0.39735838,   0.0406369 ,  -0.1480521 ,
          -0.43241135]]),
 array([[  1.95072059, -12.78536768,  -0.41602502,  10.10665721,
           1.6393452 ],
        [  4.50680528,  -3.18696565,   0.0287035 ,  -4.67659922,
           3.02918543],
        [  0.28237592,   1.53371787,  -0.86981716,  -1.20104217,
           0.99850901],
        [  0.29117683,   8.47578605,  -0.80460956,  -0.54785576,
          -0.44532903]])]